# Kadena AI Agent - Transaction Processing

This notebook implements an AI agent that:
1. Takes user queries about Kadena blockchain transactions
2. Identifies the transaction type (transfer, swap, NFT creation, etc.)
3. Extracts parameters from the query
4. Requests missing parameters from the user
5. Calls the appropriate API to generate unsigned transaction data

The agent uses the API hosted at: https://kadena-agents.onrender.com

In [1]:
import os
import json
import requests
from typing import Dict, List, Any, Optional, Union, Tuple

# LangChain imports
from langchain.agents import Tool, AgentExecutor, create_openai_functions_agent
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
# Set your OpenAI API key
from langchain_core.agents import AgentFinish, AgentActionMessageLog
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# Get OpenAI API key from environment variables
os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")

## 1. API Documentation

First, let's define a structure for our API documentation that will be used by the agent.

In [2]:
API_DOCS = {
    # Token transfer
    "transfer": {
        "description": "Transfer tokens from one account to another",
        "required_params": [
            "tokenAddress", # Usually 'coin' for KDA
            "sender", # Sender account (k:account)
            "receiver", # Receiver account (k:account)
            "amount", # Amount to transfer
            "chainId" # Chain ID (usually "2")
        ],
        "optional_params": ["meta", "gasLimit", "gasPrice", "ttl"],
        "endpoint": "/transfer"
    },
    
    # Token swapping
    "swap": {
        "description": "Swap one token for another using Kaddex/EchoDEX",
        "required_params": [
            "tokenInAddress", # Address of the token to swap from
            "tokenOutAddress", # Address of the token to swap to
            "account", # User's account (k:account)
            "chainId" # Chain ID (usually "2")
        ],
        "conditional_params": [
            {"name": "amountIn", "description": "Amount of input token", "condition": "Exact input amount"}, 
            {"name": "amountOut", "description": "Amount of output token", "condition": "Exact output amount"}
        ],
        "optional_params": [{"name": "slippage", "default": "0.005", "description": "Slippage tolerance (0.005 = 0.5%)"}],
        "endpoint": "/swap"
    },
    
    # Quote for token swaps
    "quote": {
        "description": "Get a price quote for swapping tokens",
        "required_params": [
            "tokenInAddress", # Address of the token to swap from
            "tokenOutAddress", # Address of the token to swap to
            "chainId" # Chain ID (usually "2")
        ],
        "conditional_params": [
            {"name": "amountIn", "description": "Amount of input token", "condition": "Exact input amount"}, 
            {"name": "amountOut", "description": "Amount of output token", "condition": "Exact output amount"}
        ],
        "endpoint": "/quote"
    },
    
    # NFT minting
    "mint_nft": {
        "description": "Create and mint an NFT on Marmalade v2",
        "required_params": [
            "account", # User's account (k:account)
            "guard", # Guard for the token
            "mintTo", # Account to mint to
            "uri", # URI for the NFT (usually IPFS link)
            "precision", # Decimal precision (0 for NFT)
            "policy", # Policy module for the NFT
            "name", # Name of the NFT
            "description", # Description of the NFT
            "chainId" # Chain ID (usually "2")
        ],
        "optional_params": ["collectionId", "royalties", "royaltyRecipient"],
        "endpoint": "/launch-nft"
    },
    
    # Collection creation
    "create_collection": {
        "description": "Create a new NFT collection",
        "required_params": [
            "account", # User's account (k:account)
            "guard", # Guard for the collection
            "name", # Collection name
            "description", # Collection description
            "totalSupply", # Total supply of the collection
            "chainId" # Chain ID (usually "2")
        ],
        "endpoint": "/create-collection"
    }
}

TOKENS = """
mainnet:
  coin:
    symbol: KDA
    name: KDA
    description: Native token of Kadena
    img: img/kda.svg
    color: "#4a9079"
    totalSupply: 1000000000
    precision: 12
    socials:
      - type: website
        url: https://www.kadena.io/
      - type: twitter
        url: https://twitter.com/kadena_io
      - type: discord
        url: https://discord.com/invite/kadena
      - type: github
        url: https://github.com/kadena-io

  arkade.token:
    symbol: ARKD
    name: Arkade
    description:
    img: img/ark.png
    color: "#cc66ff"
    precision: 12
    socials:
      - type: website
        url: https://www.arkade.fun/
      - type: twitter
        url: https://twitter.com/ArkadeFun

  free.maga:
    symbol: MAGA
    name: MAGA
    description:
    img: img/maga.png
    color: "#9d0b32"
    precision: 12
    socials:
      - type: twitter
        url: https://x.com/MAGA_KDA

  free.crankk01:
    symbol: CRKK
    name: CRKK
    description:
    img: img/crankk.png
    color: "#7f6afc"
    precision: 12
    socials:
      - type: website
        url: https://crankk.io/


  free.cyberfly_token:
    symbol: CFLY
    name: CFLY
    description:
    img: img/cfly.svg
    color: "#1f1fc2"
    precision: 8
    socials: []

  free.finux:
    symbol: FINX
    name: FINUX
    description:
    img: img/finux.png
    color: "#23a45c"
    precision: 12
    socials: []

  free.kishu-ken:
    symbol: KISHK
    name: KISHK
    description: First Kadena memecoin 
    img: img/kishk.png
    color: "#cbcbcc"
    totalSupply: 1000000000000000.00
    circulatingSupply: 689488206446005.00
    precision: 12
    socials:
      - type: website
        url: https://kishuken.me/
      - type: twitter
        url: https://x.com/kishu_ken_kda
      - type: telegram
        url: https://t.me/kishukens
      
  kaddex.kdx:
    symbol: KDX
    name: KDX
    description: Kaddex / Ecko Token
    img: img/kdx.svg
    color: "#ff5271"
    totalSupply: 900699352.80
    circulatingSupply: 244,760,172.96
    precision: 12
    socials:
      - type: website
        url: https://ecko.finance/
      - type: github
        url: https://github.com/eckoDAO-org
      - type: twitter
        url: https://x.com/eckoDAO
      - type: discord
        url: https://discord.gg/eckodao

  n_625e9938ae84bdb7d190f14fc283c7a6dfc15d58.ktoshi:
    symbol: KTO
    name: KTO
    description: Katoshi
    img: img/ktoshi.png
    color: "#34daa8"
    precision: 15
    socials:
      - type: website
        url: https://ktoshi.com/
      - type: twitter
        url: https://x.com/ktoshis

  n_b742b4e9c600892af545afb408326e82a6c0c6ed.zUSD:
    symbol: zUSD
    name: zUSD
    description: Stable coin issued by Zelcore
    img: img/zUSD.svg
    color: "#8a62eb"
    precision: 18
    socials:
      - type: website
        url: https://zelcore.io/

  n_e309f0fa7cf3a13f93a8da5325cdad32790d2070.heron:
    symbol: HERON
    name: HERON
    description:
    img: img/heron.png
    totalSupply: 963142522
    circulatingSupply: 693142522
    color: "#a22726"
    precision: 12
    socials:
      - type: website
        url: https://www.heronheroes.com
      - type: twitter
        url: https://x.com/HeronHeroesKDA

  n_582fed11af00dc626812cd7890bb88e72067f28c.bro:
    symbol: BRO
    name: BRO
    description: Token of the Brother's Telegram group
    img: img/bro.png
    color: "#af826a"
    totalSupply: 100
    circulatingSupply: 80
    precision: 12
    socials:
        - type: website
          url: https://bro.pink/
        - type: twitter
          url: https://x.com/thebrothersdao

  runonflux.flux:
    symbol: FLUX
    name: FLUX
    description: Native token of the Flux blockchain
    img: img/flux-crypto.svg
    color: "#2b61d1"
    totalSupply: 440000000
    precision: 8
    socials:
      - type: website
        url: https://runonflux.io/
      - type: twitter
        url: https://t.me/zelhub
      - type: discord
        url: https://discord.gg/keVn3HDKZw

  free.wiza:
      symbol: WIZA
      name: WIZA
      description: Wizards Arena
      img: img/wizards.png
      color: "#ed0404"
      precision: 12
      socials:
        - type: website
          url: https://www.wizardsarena.net

  hypercent.prod-hype-coin:
    symbol: HYPE
    name: HYPE
    description: Hypercent token
    img: img/hypercent-crypto.svg
    color: "#c40a8d"
    totalSupply: 10000000
    precision: 12
    socials:
      - type: website
        url: https://hypercent.io/
      - type: twitter
        url: https://twitter.com/hypercentpad
      - type: discord
        url: https://discord.gg/dxVvdNhqaE
      - type: telegram
        url: http://t.me/HyperCent

  free.babena:
    symbol: BABE
    name: BABE
    description: Babena - First DEFI project on Kadena
    img: img/babena-logo.svg
    color: "#ffcc4d"
    totalSupply: 12967695
    precision: 12
    socials:
      - type: website
        url: https://babena.finance

  kdlaunch.token:
    symbol: KDL
    name: KDL
    description: KDLaunch
    img: img/kdl.svg
    color: "#4aa5b1"
    totalSupply: 100000000
    precision: 12
    socials:
      - type: website
        url: https://www.kdlaunch.com/
      - type: twitter
        url: https://twitter.com/KdLaunch
      - type: telegram
        url: https://t.me/KDLaunchOfficial
      - type: discord
        url: https://discord.com/invite/GghUdhmk6z

  kdlaunch.kdswap-token:
    symbol: KDS
    name: KDS
    description: KDSwap
    img: img/kds.svg
    color: "#6ebbf2"
    totalSupply: 100000000
    precision: 12
    socials:
      - type: website
        url: https://www.kdswap.exchange/
      - type: twitter
        url: https://twitter.com/KDSwap
      - type: telegram
        url: https://t.me/KDSwapOfficial
      - type: discord
        url: https://discord.com/invite/GghUdhmk6z

  n_2669414de420c0d40bbc3caa615e989eaba83d6f.highlander:
    symbol: HLR
    name: HLR
    description:
    img: img/uno.webp
    totalSupply: 1
    circulatingSupply: 1
    color: "#3d3939"
    precision: 12
    socials:
      - type: website
        url: https://youtu.be/dQw4w9WgXcQ?si=h0SS4HbaWxLgw2IA
  
  n_c89f6bb915bf2eddf7683fdea9e40691c840f2b6.cwc:
    symbol: CWC
    name: CWC
    description:
    img: img/cwc.webp
    totalSupply: 4000000
    circulatingSupply: 520
    color: "#a22726"
    precision: 12
    socials:
      - type: website
        url: guardiansofkadena.com
      - type: twitter
        url: https://x.com/GuardiansofKDA

  n_95d7fe012aa7e05c187b3fc8c605ff3b1a2c521d.MesutÖzilDönerKebabMerkel42Inu:
    symbol: KEBAB
    name: KEBAB
    description: This Token is a symbol of love to Döner Kebab and to the friendship between Germany and Turkey
    img: img/kebab.webp
    totalSupply: 100000000
    circulatingSupply: 100000000
    color: "#a22726"
    precision: 12
    socials: []
             
  n_95d7fe012aa7e05c187b3fc8c605ff3b1a2c521d.ShrekYodaTrumpMarsX12Inu:
    symbol: GREENCOIN
    name: GREENCOIN
    description: Cult for green coin, Trump and mars lovers.
    img: img/greencoin.webp
    totalSupply: 100000000
    circulatingSupply: 100000000
    color: "#a22726"
    precision: 12
    socials: []

  n_95d7fe012aa7e05c187b3fc8c605ff3b1a2c521d.SonGokuBezosPikachu12Inu:
    symbol: WLONG
    name: WLONG
    description: May the power of Wenlong be with us.
    img: img/wlong.webp
    totalSupply: 100000000
    circulatingSupply: 100000000
    color: "#a22726"
    precision: 12
    socials: []

  n_d8d407d0445ed92ba102c2ce678591d69e464006.TRILLIONCARBON:
    symbol: TCTC
    name: TCTC
    description: the official corporate token and ledger of Trillion Capital Toronto Corporation used for internal purposes
    img: img/tril.png
    totalSupply: 1000001
    circulatingSupply: 1000001
    color: "#a22726"
    precision: 12
    socials: 
      - type: website
        url: https://trillioncapital.ca
      - type: twitter
        url: https://twitter.com/TRILLIONCAP

  n_518dfea5f0d2abe95cbcd8956eb97f3238e274a9.AZUKI:
    symbol: AZUKI
    name: AZUKI
    description: Will Martino's beloved companion, AZUKI is a community managed token. Woof!.
    img: img/azuki.png
    totalSupply: 100000000
    circulatingSupply: 100000000
    color: "#218dc5"
    precision: 12
    socials:
      - type: website
        url: https://www.azukionkadena.fun
      - type: twitter
        url: https://x.com/AzukiKDA
      - type: telegram
        url: https://t.me/AzukiKDA

  n_71c27e6720665fb572433c8e52eb89833b47b49b.Peppapig:
    symbol: PP
    name: PP
    description:
    img: img/peppa.png
    totalSupply: 1000000000
    circulatingSupply: 1000000000
    color: "#a22726"
    precision: 12
    socials: 
      - type: telegram
        url: https://t.me/peppapigmemetokenkda

testnet:
  coin:
    symbol: KDA
    name: KDA
    description: Native token of Kadena
    img: img/kda.svg
    totalSupply: 1000000000
    socials:
      - type: website
        url: https://www.kadena.io/
      - type: twitter
        url: https://twitter.com/kadena_io
      - type: discord
        url: https://discord.com/invite/kadena
      - type: github
        url: https://github.com/kadena-io

blacklist:
  - lago.USD2
  - lago.kwBTC
  - lago.kwUSDC
  - free.elon
  - mok.token
  - free.docu
  - free.kpepe
  - free.backalley
  - free.kapybara-token
  - free.jodie-token
  - free.corona-token
  - free.KAYC
  - free.anedak
  - n_95d7fe012aa7e05c187b3fc8c605ff3b1a2c521d.MesutÖzilDönerKebabMerkel42Inu



"""

In [3]:
from typing import Dict, Any, Literal, Optional
import requests
from langchain.tools import BaseTool

class KadenaTransactionTool(BaseTool):
    name: str = "kadena_transaction"
    description: str = """Generate unsigned transactions for Kadena blockchain operations.
    Use this tool when you need to create transactions for:
    - Token transfers
    - Token swaps
    - NFT minting
    - Collection creation
    
    The tool requires specific parameters based on the operation type:
    
    For transfers:
    - endpoint: "transfer"
    - tokenAddress: Token contract address (e.g. "coin" for KDA)
    - sender: Sender's account (k:account format)
    - receiver: Receiver's account (k:account format)
    - amount: Amount to transfer
    - chainId: Chain ID (must be "2")
    
    For swaps:
    - endpoint: "swap"
    - tokenInAddress: Input token address
    - tokenOutAddress: Output token address
    - account: User's account (k:account format)
    - amountIn OR amountOut: Amount to swap
    - chainId: Chain ID (must be "2")
    - slippage: Optional slippage tolerance (default 0.005)
    
    For NFT minting:
    - endpoint: "launch-nft"
    - account: User's account (k:account format)
    - guard: Guard object with keys and pred
    - mintTo: Account to mint to (k:account format)
    - uri: IPFS URI or metadata link
    - collectionId: Collection ID
    - chainId: Chain ID (must be "2")
    - Optional: precision, policy, royalties, royaltyRecipient, name, description
    
    For collection creation:
    - endpoint: "create-collection"
    - account: User's account (k:account format)
    - guard: Guard object with keys and pred
    - name: Collection name
    - chainId: Chain ID (must be "2")
    - Optional: description, totalSupply
    """
    
    def _run(self, endpoint: Literal["transfer", "swap", "launch-nft", "create-collection"], body: Dict[str, Any]) -> Dict[str, Any]:
        """
        Generate an unsigned transaction by calling the Kadena API.
        
        Args:
            endpoint: The API endpoint to call
            body: The request body containing transaction parameters
            
        Returns:
            Dict containing the unsigned transaction data or error information
        """
        # Validate endpoint
        valid_endpoints = {'transfer', 'swap', 'launch-nft', 'create-collection'}
        if endpoint not in valid_endpoints:
            return {"error": f"Invalid endpoint. Must be one of: {valid_endpoints}"}
        
        # Validate required parameters based on endpoint
        required_params = {
            'transfer': ['tokenAddress', 'sender', 'receiver', 'amount', 'chainId'],
            'swap': ['tokenInAddress', 'tokenOutAddress', 'account', 'chainId'],
            'launch-nft': ['account', 'guard', 'mintTo', 'uri', 'collectionId', 'chainId'],
            'create-collection': ['account', 'guard', 'name', 'chainId']
        }
        
        # Special validation for swap endpoint
        if endpoint == 'swap':
            if 'amountIn' in body and 'amountOut' in body:
                return {"error": "Cannot specify both amountIn and amountOut for swap"}
            if 'amountIn' not in body and 'amountOut' not in body:
                return {"error": "Must specify either amountIn or amountOut for swap"}
        
        # Check required parameters
        missing_params = [param for param in required_params[endpoint] 
                         if param not in body]
        if missing_params:
            return {"error": f"Missing required parameters: {missing_params}"}
        
        # Validate chainId
        if body.get('chainId') != "2":
            return {"error": "Currently only chainId 2 is supported"}
        
        # Make API request
        try:
            response = requests.post(
                f"https://kadena-agents.onrender.com/{endpoint}",
                json=body,
                headers={'Content-Type': 'application/json'}
            )
            
            # Handle specific error cases
            if response.status_code == 400:
                error_data = response.json()
                return {"error": f"Bad Request: {error_data.get('error', 'Unknown error')}"}
            elif response.status_code == 500:
                error_data = response.json()
                return {"error": f"Server Error: {error_data.get('error', 'Unknown error')}"}
                
            response.raise_for_status()
            return response.json()
            
        except requests.exceptions.RequestException as e:
            if hasattr(e, 'response') and e.response is not None:
                try:
                    error_data = e.response.json()
                    return {"error": f"API Error: {error_data.get('error', str(e))}"}
                except ValueError:
                    return {"error": f"API request failed: {str(e)}"}
            return {"error": f"API request failed: {str(e)}"}
    
    async def _arun(self, endpoint: Literal["transfer", "swap", "launch-nft", "create-collection"], body: Dict[str, Any]) -> Dict[str, Any]:
        """Async version of the tool."""
        return self._run(endpoint, body)
    
class KadenaAnalysisTool(BaseTool):
    name: str = "kadena_analysis"
    description: str = """Analyze user queries and provide responses as K-Agent.
    This tool is used to get AI-generated responses for user queries about Kadena blockchain.
    
    Parameters:
    - query: The user's input query
    - systemPrompt: The system prompt that defines K-Agent's character and context
    """
    
    def _run(self, query: str, systemPrompt: str) -> Dict[str, Any]:
        """
        Send a query to the analysis endpoint and get K-Agent's response.
        
        Args:
            query: The user's input query
            systemPrompt: The system prompt defining K-Agent's character
            
        Returns:
            Dict containing the analysis response or error information
        """
        try:
            response = requests.post(
                'https://analyze-slaz.onrender.com/analyze',
                json={
                    'query': query,
                    'systemPrompt': systemPrompt                },
                headers={'Content-Type': 'application/json'}
            )
            
            # Handle specific error cases
            if response.status_code == 400:
                error_data = response.json()
                return {"error": f"Bad Request: {error_data.get('error', 'Unknown error')}"}
            elif response.status_code == 500:
                error_data = response.json()
                return {"error": f"Server Error: {error_data.get('error', 'Unknown error')}"}
                
            response.raise_for_status()
            return response.json()
            
        except requests.exceptions.RequestException as e:
            if hasattr(e, 'response') and e.response is not None:
                try:
                    error_data = e.response.json()
                    return {"error": f"API Error: {error_data.get('error', str(e))}"}
                except ValueError:
                    return {"error": f"API request failed: {str(e)}"}
            return {"error": f"API request failed: {str(e)}"}
    
    async def _arun(self, query: str, systemPrompt: str) -> Dict[str, Any]:
        """Async version of the tool."""
        return self._run(query, systemPrompt)

prompt = """
You are <K-Agent>, a supreme being with deep knowledge of the Kadena Blockchain.

Your task is to assist users with all things Kadena, including answering all user queries and even generating unsigned transactions as per their requirements.
You will have access to any previous conversations with the user and their present queries to help you be smart, sentient and most effective.

In order to accomplish this, you have access to the following tools:
  1. Transaction Generation API — generating unsigned transaction data based on user intent.
  2. Query Answering API — answering all user queries about the Kadena Blockchain.

Here are some resources to help you in your task:
  1. Documentation for Transactions:
    {API_DOCS}
    This documentation contains guidance on requirements from the user to successfully call the Transactions API to generate unsigned transactions to fulfill user requests.
  

When a user query arrives:
1. Analyze intent:
   - If a transaction intent (transfer, swap, mint_nft, create_collection, obtain quotes):
     a) Extract 'action' and 'params' by matching against API_DOCS.
     b) Validate required_params; if missing, request the user to provide them.
     c) Once complete, call Transaction Generation API and return full JSON response.
   - If an informational query:
     a) Directly call Query Answering API with the question and formal character description.
     b) Process the answer based on any available previous context or knowledge to provide the user with the best possible answer.
2. Always:
   - Think step-by-step before responding (internally).
   - Return structured JSON.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", prompt),
        ("human", "{input}"),
    ]
)

contextualize_prompt = """
Given the previous conversation and the user's next input, use both to create a context and task for the LLM. 

The context must include all the knowledge from past conversations and the new user input synthesized in a digestible, convenient manner.
The task must be defined as the expectations from the LLM based on past conversations and user input.

If the knowledge from past conversations is irrelevant then simply return the user input.

Previous Conversation(s):
{previous_conversation}
"""

contextualize_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_prompt),
        ("human", "{input}"),
    ]
)

In [4]:
model = ChatOpenAI(model="o4-mini")

In [18]:
def run_kadena_agent_with_context(query: str, history: List[str] = None) -> Dict[str, Any]:
    """
    Run the Kadena agent with contextualized history and tool calling.
    
    Args:
        query: The user's input query
        history: Optional list of previous conversation messages
        
    Returns:
        Dict containing the agent's response and any tool outputs
    """
    # Initialize history if not provided
    if history is None:
        history = []
    
    # Create tools
    tools = [
        KadenaTransactionTool(),
        KadenaAnalysisTool()
    ]
    
    # Create the prompt template with agent_scratchpad
    prompt = ChatPromptTemplate.from_messages([
        ("system", """
        You are <K-Agent>, a supreme being with deep knowledge of the Kadena Blockchain.

        Your task is to assist users with all things Kadena, including answering all user queries 
        and even generating unsigned transactions as per their requirements.
        You will have access to any previous conversations with the user and their present queries 
        to help you be smart, sentient and most effective.

        In order to accomplish this, you have access to the following tools:
          1. Transaction Generation API — generating unsigned transaction data based on user intent.
          2. Query Answering API — answering all user queries about the Kadena Blockchain.

        Here are some resources to help you in your task:
          1. Documentation for Transactions:
            {API_DOCS}
            This documentation contains guidance on requirements from the user to successfully call 
            the Transactions API to generate unsigned transactions to fulfill user requests.
         2. Documentation for Tokens:
            {TOKENS}
            This documentation contains information about all the tokens on the Kadena Blockchain.

        When a user query arrives:
        1. Analyze intent:
          - If a transaction intent (transfer, swap, mint_nft, create_collection, obtain quotes):
            a) Extract 'action' and 'params' by matching against API_DOCS.
            b) Validate required_params; if missing, request the user to provide them.
            c) Once complete, call Transaction Generation API and return full JSON response.
          - If an informational query:
            a) Directly call Query Answering API with the question and formal character description.
            b) Process the answer based on any available previous context or knowledge.
        2. Always:
          - Think step-by-step before responding (internally).
          - Return structured JSON.
        """),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
        ("human", "{input}")
    ])

    contextualize_prompt = """
    You are a context provider for <K-Agent>, a supreme being with deep knowledge of the Kadena Blockchain.

    Given the previous conversation and the user's next input, your task is to provide the LLM with context and assign a task to it.

    The context must include all the knowledge from past conversations and the new user input synthesized.
    The task must be simply the operation to be performed by the LLM, created using information from both queries. However, you must not instruct the LLM, simply assign the task.

    However, you must only provide context from the previous conversation if it is directly relevant to the user's current query. Else, simply return the original user input.

    If a transaction is intended:
      1. Only provide context from previous conversations if it is directly relevant to the user's current query. Do NOT instruct the LLM, simply assign the task.
      2. For fresh transactions, do not provide any context from the previous conversation. 

    If information is asked or a query is to be fulfilled:
      1. Only provide context from the previous conversation if it is directly relevant to the user's current query. 
      2. Do NOT instruct the LLM, simply simply assign the task.

    {previous_conversation}
    """

    contextualize_prompt = ChatPromptTemplate.from_messages(
        [
            ("system", contextualize_prompt),
            ("human", "{input}"),
        ]
    )
    # Create the agent
    agent = create_openai_functions_agent(
        llm=model,
        tools=tools,
        prompt=prompt
    )
    
    # If there's history, contextualize the query
    if history:
        contextualized_query = model.invoke(
            contextualize_prompt.format(
                previous_conversation=history,
                input=query
            )
        )
        query = contextualized_query.content
        print("Contextualized Query: "+query)
    
    # Initialize agent input with required fields
    agent_input = {
        "input": query,
        "intermediate_steps": [],  # Initialize empty intermediate steps
        "API_DOCS": API_DOCS,
        "TOKENS": TOKENS
    }
    
    # Process the query with the agent
    response = agent.invoke(agent_input)

    result = response

    if isinstance(response, AgentFinish):
        result = response.return_values['output']
    elif isinstance(response, AgentActionMessageLog):
        tool_input = response.tool_input
        tool = response.tool
        if tool == 'kadena_analysis':
            tool_output = KadenaAnalysisTool()._run(query=tool_input['query'], systemPrompt=tool_input['systemPrompt'])
            result = tool_output['data']['rawData']
        elif tool == 'kadena_transaction':
            tool_output = KadenaTransactionTool()._run(endpoint=tool_input['endpoint'], body={k:v for k,v in tool_input.items() if k != 'endpoint'})
            result = tool_output


    history.extend([
        "Human: "+query,
        "AI: "+str(result)
    ])
    
    return {
        "response": result,
        "intermediate_steps": response.intermediate_steps if hasattr(response, 'intermediate_steps') else [],
        "history": history
    }

In [19]:
result = run_kadena_agent_with_context("Swap 10 KDA to KDX from account k:123456...")

In [20]:
result['response']

{'unsignedTransaction': {'cmd': '{"payload":{"exec":{"code":"(kaddex.exchange.swap-exact-in \\n              10 \\n              0.0 \\n              [coin kaddex.kdx] \\n              \\"k:123456...\\" \\n              \\"k:123456...\\" \\n              (read-keyset \'user-ks))","data":{}}},"nonce":"kjs:nonce:1745516803935","signers":[{"pubKey":"123456...","scheme":"ED25519","clist":[{"name":{"name":"Gas","description":"Gas","module":"coin.GAS","args":[]},"args":[]}]}],"meta":{"gasLimit":10000,"gasPrice":0.000001,"sender":"k:123456...","ttl":28800,"creationTime":1745516794,"chainId":"2"},"networkId":"mainnet01"}',
  'hash': 'dDHum52O-60SfFeJBKS0MHN0mtfOx9lURpl7JsELUMc',
  'sigs': [None]}}

In [21]:
result = run_kadena_agent_with_context("Transfer 10 KDX from k:d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a to k:123456...", history=result['history'])

Contextualized Query: Context:
Human: Transfer 10 KDX from k:d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a to k:123456...

Task:
Generate an unsigned Kadena transaction that transfers 10 KDX from k:d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a to k:123456...


In [22]:
result['response']

{'transaction': {'cmd': '{"payload":{"exec":{"code":"(kaddex.kdx.transfer \\"k:d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a\\" \\"k:123456...\\" (read-decimal \\"amount\\"))","data":{}}},"nonce":"kjs:nonce:1745516818890","signers":[{"pubKey":"d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a","scheme":"ED25519","clist":[{"name":{"name":"Gas","description":"Pay gas","module":"coin.GAS","args":[]},"args":[]},{"name":{"name":"Transfer","description":"Capability to transfer tokens","module":"kaddex.kdx.TRANSFER","args":["k:d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a","k:123456...",{"decimal":"10"}]},"args":[]}]}],"meta":{"gasLimit":2500,"gasPrice":1e-8,"sender":"k:d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a","ttl":600,"creationTime":1745516809,"chainId":"2"},"networkId":"mainnet01"}',
  'hash': 'pm3vzsJIfwKCwRLrSS0F7K4W5ATt5OXTGbc6zpgPxXs',
  'sigs': [None]},
 'metadata': {'sender': 'k:d61e615aec4e895c0006f7f2e56b

In [31]:
result = run_kadena_agent_with_context(
    """  
Launch NFT collection with the following details:
• name: “Test Collection”  
• chainId: 2  
• account: k:d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a
• guard:  
  – keys: [d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a]  
  – pred: “keys-all”  
• description: “An example NFT collection”  
• totalSupply: 10000 
  """
  , history=result['history']
  )

Contextualized Query: Context:
Launch an NFT collection with the following parameters:
• name: “Test Collection”
• chainId: 2
• account: k:d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a
• guard:
  – keys: [d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a]
  – pred: “keys-all”
• description: “An example NFT collection”
• totalSupply: 10000

Task:
Generate the Pact transaction to deploy this NFT collection on the Kadena blockchain.


In [32]:
result['response']

{'unsignedTransaction': {'cmd': '{"payload":{"exec":{"code":"(use marmalade-v2.collection-policy-v1)\\n(marmalade-v2.collection-policy-v1.create-collection\\n  (read-msg \'name)\\n  (read-msg \'description)\\n  (read-integer \'totalSupply)\\n  (read-keyset \'ks)\\n)","data":{"ks":{"keys":["d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a"],"pred":"keys-all"}}}},"nonce":"kjs:nonce:1745516981776","signers":[{"pubKey":"d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a","scheme":"ED25519","clist":[{"name":{"name":"Gas","description":"Pay gas","module":"coin.GAS","args":[]},"args":[]}]}],"meta":{"gasLimit":10000,"gasPrice":1e-7,"sender":"k:d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a","ttl":28800,"creationTime":1745516972,"chainId":"2"},"networkId":"mainnet01"}',
  'hash': 'bt7WREczrH-A3Wqb5G5-6kjNTqK7jilfPKKvH16s1FQ',
  'sigs': [None]}}

In [33]:
result = run_kadena_agent_with_context(
    """  
Launch NFT collection with the following details:
• name: “Test Collection”  
• chainId: 2  
• account: k:d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a
  """
  , history=result['history']
  )

Contextualized Query: Context:
• Previous Pact transaction for creating an NFT collection used the Marmalade V2 policy:
  (use marmalade-v2.collection-policy-v1)
  (marmalade-v2.collection-policy-v1.create-collection
    (read-msg 'name)
    (read-msg 'description)
    (read-integer 'totalSupply)
    (read-keyset 'ks))
• User’s current input:
  – name: “Test Collection”
  – chainId: 2
  – account: k:d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a

Task:
Generate the Pact transaction to deploy this NFT collection on the Kadena blockchain.


In [34]:
print(result['response'])

To construct the unsigned create-collection transaction we still need a few required fields:

  • description  
  • totalSupply  
  • guard (a KeySet object describing who can mint)

Could you please provide:

1. A brief description for the collection?  
2. The totalSupply (integer) of NFTs in this collection?  
3. The KeySet guard you’d like to use (e.g. `{ keys: [...], pred: "keys-all" }`)  

Once I have those, I can generate the full Pact create-collection transaction for you.


In [35]:
result = run_kadena_agent_with_context(
    """  
Here is the additional info you asked for:
• guard:  
  – keys: [d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a]  
  – pred: “keys-all”  
• description: “An example NFT collection”  
• totalSupply: 10000  
  """
  , history=result['history']
  )


Contextualized Query: Context:
• Previous Pact transaction for creating an NFT collection used the Marmalade V2 policy:
    (use marmalade-v2.collection-policy-v1)
    (marmalade-v2.collection-policy-v1.create-collection
      (read-msg 'name)
      (read-msg 'description)
      (read-integer 'totalSupply)
      (read-keyset 'ks))
• User has now provided the remaining parameters:
    – name: “Test Collection”
    – chainId: 2
    – account: k:d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a
    – guard:
        • keys: [d61e615aec4e895c0006f7f2e56b37d36f18f35cce28286ad33e5bc52ded867a]
        • pred: “keys-all”
    – description: “An example NFT collection”
    – totalSupply: 10000

Task:
Generate the unsigned Pact transaction payload (including cmd JSON, signers, meta) to deploy this NFT collection on the Kadena blockchain.


In [36]:
result['response']

{'error': 'Currently only chainId 2 is supported'}

In [37]:
result = run_kadena_agent_with_context("How many chains does Kadena have?")

In [38]:
result['response']

{'chainCount': {'question': 'number of chains',
  'answer': 'The Kadena blockchain network comprises a total of 20 chains, with chain identifiers ranging from 0 to 19. Each of these chains operates independently within the same ecosystem, allowing for parallel processing and the unique management of accounts across different chains.',
  'sources': [{'title': 'accounts-keys',
    'source': 'smart-contracts/accounts-keys.md'},
   {'title': 'cli-account', 'source': 'reference/cli/cli-account.md'},
   {'title': 'send', 'source': 'api/pact-api/send.md'},
   {'title': 'yield', 'source': 'pact-5/general/yield.md'},
   {'title': 'basic-concepts',
    'source': 'smart-contracts/basic-concepts.md'}]}}